In [1]:
import pandas as pd
raw_sample = pd.read_csv('/kaggle/input/raw_sample.csv')
ad_feature = pd.read_csv('/kaggle/input/ad_feature.csv')
user_profile = pd.read_csv('/kaggle/input/user_profile.csv')

In [4]:
userBehavior = pd.read_csv('/kaggle/input/UserBehavior.csv')

In [6]:
userBehavior.columns = ['user_id', 'item_id', 'category_id', 'behavior_type', 'timestamp']

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset

# Charger les données Ads
ads_data = pd.read_csv("/kaggle/input/raw_sample.csv", usecols=["user", "adgroup_id", "nonclk"])
ads_data.rename(columns={"user": "user_id"}, inplace=True)

In [9]:
# Charger toutes les colonnes et renommer
userBehavior = pd.read_csv("/kaggle/input/UserBehavior.csv")
userBehavior.columns = ['user_id', 'item_id', 'category_id', 'behavior_type', 'timestamp']

# Sélectionner les colonnes nécessaires
ecommerce_data = userBehavior[["user_id", "item_id", "behavior_type"]]

In [10]:
ecommerce_data_reduite = ecommerce_data.sample(frac=0.01, random_state=42)

raw_sample :  (1001508, 3)


,user_id,item_id,behavior_type
43314727,237858,1481482,pv
88035904,44200,3916487,pv
38200464,920479,1955086,pv
83578910,239462,2698818,pv
78996463,947273,3511372,pv
...,...,...,...
96241465,821287,2192371,pv
84258940,270792,131317,pv
4871299,304011,1237577,pv
6745089,388702,535159,pv


In [12]:
ads_data_reduite_non_equilibre = ads_data.sample(frac=0.02, random_state=42)

raw_sample :  (531159, 3)


,user_id,adgroup_id,nonclk
22276667,506615,570116,1
4373967,846189,554740,1
4187248,601204,542200,0
3102892,675220,450876,1
9831068,756683,160239,1
...,...,...,...
20272257,670761,389825,1
14133863,613111,608224,1
7561579,987222,734790,1
15111726,139731,656398,1


In [14]:
from sklearn.utils import resample

# Diviser les données en classes majoritaires et minoritaires
majority_class = ads_data_reduite_non_equilibre[ads_data_reduite_non_equilibre["nonclk"] == 1]
minority_class = ads_data_reduite_non_equilibre[ads_data_reduite_non_equilibre["nonclk"] == 0]

# Taille souhaitée pour la classe minoritaire (un quart de la classe majoritaire)
target_minority_size = len(majority_class) // 4

# Ajuster la classe minoritaire
if len(minority_class) >= target_minority_size:
    # Sous-échantillonner la classe minoritaire
    minority_resampled = minority_class.sample(
        n=target_minority_size,
        replace=False,  # Pas de duplication
        random_state=42
    )
else:
    # Réduire la taille de la classe majoritaire pour respecter le ratio
    majority_class = majority_class.sample(
        n=len(minority_class) * 4,  # Adapter la taille de la classe majoritaire
        replace=False,
        random_state=42
    )
    minority_resampled = minority_class  # Garder la classe minoritaire telle quelle

# Combiner les classes ajustées
ads_data_reduite = pd.concat([majority_class, minority_resampled])

nonclk
1    109344
0     27336
Name: count, dtype: int64


In [16]:
ads_data_reduite = ads_data_reduite.drop_duplicates()

In [19]:
# Étape 1 : Combiner les IDs utilisateur
all_user_ids = set(ads_data_reduite["user_id"].unique()).union(ecommerce_data_reduite["user_id"].unique())
user_id_mapping = {id_: idx for idx, id_ in enumerate(all_user_ids)}

# Mapper les IDs utilisateur dans les deux datasets
ads_data_reduite["user_id"] = ads_data_reduite["user_id"].map(user_id_mapping)
ecommerce_data_reduite["user_id"] = ecommerce_data_reduite["user_id"].map(user_id_mapping)

# Vérifier les dimensions après la normalisation
num_users = len(user_id_mapping)
print(f"Nombre total d'utilisateurs après combinaison : {num_users}")

# Étape 2 : Combiner les IDs item
all_item_ids = set(ads_data_reduite["adgroup_id"].unique()).union(ecommerce_data_reduite["item_id"].unique())
item_id_mapping = {id_: idx for idx, id_ in enumerate(all_item_ids)}

# Mapper les IDs item dans les deux datasets
ads_data_reduite["adgroup_id"] = ads_data_reduite["adgroup_id"].map(item_id_mapping)
ecommerce_data_reduite["item_id"] = ecommerce_data_reduite["item_id"].map(item_id_mapping)

# Vérifier les dimensions après la normalisation
num_items = len(item_id_mapping)
print(f"Nombre total d'items après combinaison : {num_items}")

# Étape 3 : Validation
assert ads_data_reduite["user_id"].max() < num_users, "Erreur : Index utilisateur hors limite dans ads_data_reduite"
assert ecommerce_data_reduite["user_id"].max() < num_users, "Erreur : Index utilisateur hors limite dans ecommerce_data_reduite"
assert ads_data_reduite["adgroup_id"].max() < num_items, "Erreur : Index item hors limite dans ads_data_reduite"
assert ecommerce_data_reduite["item_id"].max() < num_items, "Erreur : Index item hors limite dans ecommerce_data_reduite"

print("Les indices des utilisateurs et des items ont été normalisés avec succès.")

Nombre total d'utilisateurs après combinaison : 583964
Nombre total d'items après combinaison : 546895
Les indices des utilisateurs et des items ont été normalisés avec succès.


In [31]:
# Mapper les types de comportement en deux classes (binaire)
ecommerce_data_reduite['behavior_type'] = ecommerce_data_reduite['behavior_type'].map({
    "buy": 1,  # Achat
    "cart": 1,  # Ajout au panier
    "fav": 1,  # Favoris
    "pv": 0    # Vue de page
})

In [34]:
# Vérifiez que les IDs utilisateur et item sont bien mappés dans le dataset cible
ecommerce_data_reduite["user_id"] = ecommerce_data_reduite["user_id"].map(user_id_mapping)
ecommerce_data_reduite["item_id"] = ecommerce_data_reduite["item_id"].map(item_id_mapping)

In [36]:
# Remplacer les NaN dans 'user_id' et 'item_id' par une valeur par défaut (par exemple, 0)
ecommerce_data_reduite['user_id'] = ecommerce_data_reduite['user_id'].fillna(0).astype(int)
ecommerce_data_reduite['item_id'] = ecommerce_data_reduite['item_id'].fillna(0).astype(int)

# Si 'behavior_type' contient des NaN, remplissez-les avec une catégorie par défaut (par exemple, 0)
ecommerce_data_reduite['behavior_type'] = ecommerce_data_reduite['behavior_type'].fillna(0).astype(float)

In [52]:
ecommerce_data['user_id'] = ecommerce_data['user_id'].astype('int64')
ecommerce_data['item_id'] = ecommerce_data['item_id'].astype('int64')
ecommerce_data_reduite['behavior_type'] = ecommerce_data_reduite['behavior_type'].astype('float32')  # Si nécessaire

<ipython-input-52-4d997d3f85b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecommerce_data['user_id'] = ecommerce_data['user_id'].astype('int64')
<ipython-input-52-4d997d3f85b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecommerce_data['item_id'] = ecommerce_data['item_id'].astype('int64')


In [56]:
ecommerce_data_reduite[['user_id', 'item_id']].to_csv('users_items')

In [59]:
ads_data_reduite[['user_id', 'adgroup_id']].to_csv('users_ads')